<a href="https://www.kaggle.com/code/dhruvjyotidas1000/vit-small?scriptVersionId=286584633" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# I trained a pretrained DeiT ViT, converted images into patch embeddings, used self-attention to model global context, and fine-tuned it using AdamW with ImageNet normalization.

In [1]:
!pip install -q timm
# timm provides all the transformers here 
#timm = pre-trained models, torch = DL ka engine

import timm, torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.0 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
path = "/kaggle/input/vegetable-image-dataset/Vegetable Images/train"
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

train = datasets.ImageFolder(path, transform=transform) # Load images from folders (folder = class)
loader = DataLoader(train, batch_size=32, shuffle=True)
classes = train.classes
#Image ko 224×224 karna image ko tensor mai convert karna and normalizing pixels with imagenet images mean and stuff

In [3]:
model = timm.create_model(
    "deit_base_patch16_224",
    pretrained=True,
    num_classes=len(classes)
)

model = model.cuda()

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = torch.nn.DataParallel(model)

# Hum DeiT model use kar rahe hain jo actually ViT ka data-efficient version hai

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Using 2 GPUs


In [6]:
opt = torch.optim.AdamW(model.parameters(), lr=3e-5)
loss_fn = nn.CrossEntropyLoss()
#Transformer ko train karne ka recommended optimizer and Standard classification loss

In [7]:
for epoch in range(5):   
    for x, y in loader:
        x, y = x.cuda(), y.cuda()
        opt.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
    print("Epoch", epoch+1, "done")
# around 15-20 mins

Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done


In [8]:
torch.save(model.module.state_dict(), "vit_veg_best.pth")

# Saving trained model weights after all 5 epochs are done

In [9]:
from PIL import Image

def predict(img_path):
    img = Image.open(img_path).convert("RGB")
    t = transform(img).unsqueeze(0).cuda()
    with torch.no_grad():
        p = model(t).softmax(1)[0]
    return classes[p.argmax().item()], float(p.max().item())


In [11]:
predict("/kaggle/input/vegetable-image-dataset/Vegetable Images/train/Tomato/0001.jpg")


('Tomato', 0.9997316002845764)

I trained a pretrained DeiT-based Vision Transformer on a custom vegetable dataset and validated the inference pipeline successfully. Next, I plan to compare frozen-backbone, partial fine-tuning, and parameter-efficient adaptation strategies

In [12]:
path = "/kaggle/input/vegetable-image-dataset/Vegetable Images/train"

transform = transforms.Compose([
    transforms.Resize((224,224)),       # Image ko ViT size me lana
    transforms.ToTensor(),               # Image → Tensor
    transforms.Normalize(                # ImageNet normalization
        [0.485,0.456,0.406],
        [0.229,0.224,0.225]
    )
])

train = datasets.ImageFolder(path, transform=transform)
loader = DataLoader(train, batch_size=64, shuffle=True)

classes = train.classes


In [17]:
model = timm.create_model(
    "deit_base_patch16_224",
    pretrained=True,     # ImageNet se seekha hua model
    num_classes=len(classes)
).cuda()
# Baseline wala model reuse mat karo.
#Hamesha fresh model for new experiment.

In [19]:
for param in model.parameters():
    param.requires_grad = False

In [21]:
for param in model.head.parameters():
    param.requires_grad = True

In [23]:
opt = torch.optim.AdamW(
    model.head.parameters(),  # sirf head update hoga
    lr=1e-3                   # head ke liye high LR
)

loss_fn = nn.CrossEntropyLoss()


learning sirf head me ho rahi

In [24]:
for epoch in range(5):
    for x,y in loader:
        x,y = x.cuda(),y.cuda()
        opt.zero_grad()
        out=model(x)
        loss=loss_fn(out,y)
        loss.backward()
        opt.step()
    print("Epochs {Epoch+1} done")    
        

Epochs {Epoch+1} done
Epochs {Epoch+1} done
Epochs {Epoch+1} done
Epochs {Epoch+1} done
Epochs {Epoch+1} done


In [25]:
torch.save(model.state_dict(), "vit_frozen_head.pth")


# Partial Fine-tuning (Last Transformer Blocks + Head)
Sirf last 2 transformer blocks + head trainable

In [26]:
model = timm.create_model(
    "deit_base_patch16_224",
    pretrained=True,
    num_classes=len(classes)
).cuda()


In [27]:
for param in model.parameters():
    param.requires_grad = False


In [29]:
# DeiT / ViT me transformer blocks model.blocks me hote hain
for block in model.blocks[-2:]:   # last 2 blocks
    for param in block.parameters():
        param.requires_grad = True


In [30]:
for param in model.head.parameters():
    param.requires_grad = True


In [31]:
opt = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4   # partial finetune = medium LR
)

loss_fn = nn.CrossEntropyLoss()


In [32]:
for epoch in range(5):
    for x, y in loader:
        x, y = x.cuda(), y.cuda()
        opt.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
    print(f"Epoch {epoch+1} done")


Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done


In [33]:
torch.save(model.state_dict(), "vit_partial_finetune.pth")


# Original + Custom NN (Custom Head)

In [34]:
model = timm.create_model(
    "deit_base_patch16_224",
    pretrained=True,
    num_classes=len(classes)  # temporary, abhi replace karenge
).cuda()


In [35]:
for param in model.parameters():
    param.requires_grad = False


In [36]:
in_features = model.head.in_features


In [37]:
model.head = nn.Sequential(
    nn.Linear(in_features, 512),  # thoda compression
    nn.ReLU(),                    # non-linearity
    nn.Dropout(0.3),              # overfitting kam
    nn.Linear(512, len(classes))  # final classes
).cuda()


In [38]:
for param in model.head.parameters():
    param.requires_grad = True


In [39]:
opt = torch.optim.AdamW(
    model.head.parameters(),
    lr=1e-3   # head ke liye high LR
)

loss_fn = nn.CrossEntropyLoss()


In [40]:
for epoch in range(5):
    for x, y in loader:
        x, y = x.cuda(), y.cuda()
        opt.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
    print(f"Epoch {epoch+1} done")


Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done


In [41]:
torch.save(model.state_dict(), "vit_custom_head.pth")


# Finetuned-LoRA (parameter-efficient fine-tuning)

ViT backbone freeze + Attention layers me LoRA adapters + sirf LoRA train

Ab LoRA kyun next hai? (simple Hinglish)

Ab tak tum weights ko directly train kar rahe the

LoRA me:

Original ViT weights ❄️ frozen

Sirf chhote adapter matrices train 🔥

Matlab:

Kam parameters

Kam GPU usage

Modern research approach

In [43]:
!pip install -q timm peft
import timm, torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

from peft import LoraConfig, get_peft_model


2025-12-16 12:47:14.576439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765889234.820277      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765889234.884726      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [45]:
path = "/kaggle/input/vegetable-image-dataset/Vegetable Images/train"

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485,0.456,0.406],
        [0.229,0.224,0.225]
    )
])

train = datasets.ImageFolder(path, transform=transform)
loader = DataLoader(train, batch_size=64, shuffle=True)

classes = train.classes


In [46]:
model = timm.create_model(
    "deit_base_patch16_224",
    pretrained=True,
    num_classes=len(classes)
)


In [49]:
lora_config = LoraConfig(
    r=8,                    # rank (small = efficient)
    lora_alpha=16,          # scaling factor
    target_modules=["qkv"], # ViT attention ke andar
    lora_dropout=0.1,
    bias="none"
)
# r = LoRA kitna powerful hoga

#qkv = attention ka heart (Query, Key, Value)

In [51]:
model = get_peft_model(model, lora_config)
model = model.cuda()
model.print_trainable_parameters()


trainable params: 294,912 || all params: 86,105,103 || trainable%: 0.3425


In [53]:
opt = torch.optim.AdamW(
    model.parameters(),   # sirf LoRA params actually update honge
    lr=1e-4
)

loss_fn = nn.CrossEntropyLoss()


In [54]:
for epoch in range(5):
    for x, y in loader:
        x, y = x.cuda(), y.cuda()
        opt.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
    print(f"Epoch {epoch+1} done")


Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done


In [55]:
torch.save(model.state_dict(), "vit_lora.pth")


# ✔ original vit
# ✔ original + custom nn
# ✔ finetune
# ✔ partial finetune
# ✔ finetuned-LoRA ← DONE

In [56]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

test_path = "/kaggle/input/vegetable-image-dataset/Vegetable Images/test"

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485,0.456,0.406],
        [0.229,0.224,0.225]
    )
])

test_data = datasets.ImageFolder(test_path, transform=transform)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

classes = test_data.classes


In [58]:
import torch

def evaluate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.cuda(), y.cuda()
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total


In [61]:
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [62]:
# MODEL LOADER ----
import timm
import torch.nn as nn

def load_vit_model(weight_path, model_type):
    model = timm.create_model(
        "deit_base_patch16_224",
        pretrained=False,
        num_classes=len(classes)
    )

    # Custom head case
    if model_type == "custom_head":
        in_features = model.head.in_features
        model.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, len(classes))
        )

    model.load_state_dict(torch.load(weight_path))
    model = model.cuda()
    return model


In [63]:
models_info = {
    "Full Fine-tune": ("vit_veg_best.pth", "normal"),
    "Frozen Head": ("vit_frozen_head.pth", "normal"),
    "Partial Fine-tune": ("vit_partial_finetune.pth", "normal"),
    "Custom Head": ("vit_custom_head.pth", "custom_head"),
}

results = []

for name, (path, mtype) in models_info.items():
    model = load_vit_model(path, mtype)
    acc = evaluate_accuracy(model, test_loader)
    params = count_trainable_params(model)

    results.append((name, acc, params))
    print(f"{name} | Accuracy: {acc:.4f} | Trainable Params: {params}")


Full Fine-tune | Accuracy: 1.0000 | Trainable Params: 85810191
Frozen Head | Accuracy: 0.9987 | Trainable Params: 85810191
Partial Fine-tune | Accuracy: 1.0000 | Trainable Params: 85810191
Custom Head | Accuracy: 0.9987 | Trainable Params: 86200079


In [64]:
from peft import LoraConfig, get_peft_model

# Fresh base model
model = timm.create_model(
    "deit_base_patch16_224",
    pretrained=False,
    num_classes=len(classes)
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["qkv"],
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)
model.load_state_dict(torch.load("vit_lora.pth"))
model = model.cuda()

lora_acc = evaluate_accuracy(model, test_loader)
lora_params = count_trainable_params(model)

print(f"LoRA | Accuracy: {lora_acc:.4f} | Trainable Params: {lora_params}")


LoRA | Accuracy: 0.9997 | Trainable Params: 294912


In [65]:
print("\nFINAL COMPARISON\n")

for r in results:
    print(f"{r[0]:20s} | Acc: {r[1]:.4f} | Params: {r[2]}")

print(f"{'LoRA':20s} | Acc: {lora_acc:.4f} | Params: {lora_params}")



FINAL COMPARISON

Full Fine-tune       | Acc: 1.0000 | Params: 85810191
Frozen Head          | Acc: 0.9987 | Params: 85810191
Partial Fine-tune    | Acc: 1.0000 | Params: 85810191
Custom Head          | Acc: 0.9987 | Params: 86200079
LoRA                 | Acc: 0.9997 | Params: 294912
